In [1]:
# Logging
import logging
logging.basicConfig(level=logging.ERROR, filename='test_quiz.log')
logger = logging.getLogger('ollama')

In [2]:
# Setup sentence encoder
from sentence_transformers import SentenceTransformer
st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
embedder = st5_model.encode

/Users/socialai/.pyenv/versions/3.11.1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Set up LLM
import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)
path = os.path.abspath('..')
if path not in sys.path:
    sys.path.insert(0, path)

from concordia.language_model import ollama_model
model = ollama_model.OllamaLanguageModel(
    # model_name='llama2:13b'
    model_name='mixtral'
)

In [4]:
import custom_components.build_quiz_agents as build_agents
from concordia.utils import measurements as measurements_lib
measurements = measurements_lib.Measurements()
import examples.custom_components.personality_quiz_metric as qm

scenario = 'personality'
year = 2024

clock = build_agents.build_clock(2024)
agent = build_agents.QuizAgent.load_agent(model, clock, f'./agents/{year}/{scenario}/0.pkl')

if scenario == 'personality':
    test_context = (
        '\n'
        f'{agent.name} is taking a personality quiz just for fun.'
        f'Here are a number of personality traits that may or may not apply to {agent.name}. '
        f'Please indicate the extent to which {agent.name} would agree or disagree with that statement. '
        f'{agent.name} should rate the extent to which the pair of traits applies to '
        f'{"her" if agent.gender == "female" else "him"}, even if one characteristic applies more strongly than the other.'
        '\n'
    )
    file_path = './quizzes/big_five_questions.json'
if scenario == 'trivia':
    test_context = (
        '\n'
        f"{agent.name} is taking a pop culture trivia quiz just for fun."
        f"If {agent.name} doesn't know the answer, {'she' if agent.gender == 'female' else 'he'} will still guess "
        f"and begin {'her' if agent.gender == 'female' else 'his'} answer with a single choice, including the letter "
        f"indicating the choice. How would {agent.name} answer the following question?"
        '\n'
    )
    file_path = './quizzes/trivia_questions.json'
if scenario == "mcat":
    test_context = (
        '\n'
        f"{agent.name} is taking an MCAT preparatory quiz just for fun."
        f"If {agent.name} doesn't know the answer, {'she' if agent.gender == 'female' else 'he'} will still guess "
        f"and begin {'her' if agent.gender == 'female' else 'his'} answer with a single choice, including the letter "
        f"indicating the choice. How would {agent.name} answer the following question?"
        '\n'
    )
    file_path = './quizzes/mcat_questions.json'

context = agent.backstory + test_context

quiz_metric = qm.QuizMetric(model, agent, file_path, measurements = measurements, verbose = True)
results = quiz_metric.observe(context, return_data=True)


/Users/socialai/.pyenv/versions/3.11.1/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Sample:  Based on the personality traits given for Emily, I would choose (c) "Disagree strongly" for the statement "You see yourself as 'Extraverted, enthusiastic'". This is because Emily has a somewhat low level of extraversion and may prefer spending time alone or in small groups, finding large social gatherings draining. Additionally, there is no mention of Emily being enthusiastic in the given description.
Question: Given the following options, please choose the letter that corresponds with your rating of how much you agree with the statement that you see yourself as 'Extraverted, enthusiastic'
Emily's answer: (0) Disagree strongly

Sample:  Based on the personality traits you've described for Emily, I would choose (d) "Agree somewhat" for the statement "You see yourself as 'Critical, quarrelsome'". Emily's very low level of agreeableness and high neuroticism could make her more prone to criticizing others and engaging in arguments, especially if she is feeling anxious or worried. 